In [1]:
# if the below variable is true,
# it also considers the reply to the msg role: "function" in the training
#conisder_function_summary = False
def process_conversations_to_training_data(conversations, conisder_function_summary = False):
    #our output json data variable
    output_data = []
    process_input = []
    # process each converstion
    for conversation in conversations["conversation"]:
        #we don't train for the input that has no functions
        if conversation['functions'] is None:
            continue

        #get role of the current msg
        cur_role = conversation['chatgptMessage']['role'].lower()
    
        if (cur_role == "assistant" and prev_role != "function") or (cur_role == "assistant" and prev_role == "function" and conisder_function_summary):
            # do an output
            temp_dict = {}
            temp_dict['input'] = process_input.copy()
            temp_dict['target'] = conversation.copy()
            output_data.append(temp_dict)
        # add processed inputs to processed
        process_input.append(conversation)
        prev_role = cur_role

    return output_data

In [15]:
## sample input

import json

with open('conversations.json', 'r') as f:
    data = json.load(f)

# sample indx for testing

print(json.dumps(data[30], indent = 4))

{
    "conversationId": "8bc0bf1b-7b07-4782-9a6a-6fe320b09517",
    "conversation": [
        {
            "chatgptMessage": {
                "role": "user",
                "content": "What is https://6963-174-64-129-70.ngrok-free.app/ about"
            },
            "functions": [
                {
                    "name": "transcodeWebPage",
                    "description": "Acquire precise webpage details or real-time search engine responses based on user-input content.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "json": {
                                "properties": {
                                    "link": {
                                        "type": "string",
                                        "description": "This parameter takes either a URL or a non-URL string. If a URL is given, the model will engage with the designated webpage to collect or interact

In [23]:
## sample output
result = process_conversations_to_training_data(data[30], True)
print(json.dumps(result[1], indent = 4))

{
    "input": [
        {
            "chatgptMessage": {
                "role": "user",
                "content": "What is https://6963-174-64-129-70.ngrok-free.app/ about"
            },
            "functions": [
                {
                    "name": "transcodeWebPage",
                    "description": "Acquire precise webpage details or real-time search engine responses based on user-input content.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "json": {
                                "properties": {
                                    "link": {
                                        "type": "string",
                                        "description": "This parameter takes either a URL or a non-URL string. If a URL is given, the model will engage with the designated webpage to collect or interact with its data. If a non-URL string is given, the model will handle i

In [27]:
# generate all data
all_data_with_summary = []
for conversation in data:
    all_data_with_summary.extend(process_conversations_to_training_data(conversation, True))

with open('development_data_with_summary.json', 'w') as f:
    json.dump(all_data_with_summary, f)

all_data_without_summary = []
for conversation in data:
    all_data_without_summary.extend(process_conversations_to_training_data(conversation, False))

with open('development_data_without_summary.json', 'w') as f:
    json.dump(all_data_without_summary, f)

In [28]:
print(len(data))
print(len(all_data_with_summary))
print(len(all_data_without_summary))

89
190
117
